In [147]:
# export
import json
import logging
import torch
import os

import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFAutoModel

from abc import ABC, abstractmethod
#from icodegen.data.core import convert_df_to_tfds, java_special_tokens, train_tokenizer
from pathlib import Path, PurePath
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast

import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler

logger = logging.getLogger()
logger.setLevel(logging.INFO)

from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, explained_variance_score
from tensorflow.keras.metrics import RootMeanSquaredError as RMSE

In [52]:
!pip install sklearn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 16.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1317 sha256=a5fb23652f711d1e0c96fc66465f91a0e644b5b8eafb45e40514ad7900237b73
  Stored in directory: /home/arnab/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [11]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

Downloading: 100%|██████████████████████████████| 570/570 [00:00<00:00, 285kB/s]
Downloading: 100%|███████████████████████████| 502M/502M [00:31<00:00, 16.5MB/s]
2022-05-05 10:13:00.646680: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 10:13:00.647625: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-8.0/lib64
2022-05-05 10:13:00.647863: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-8.0/lib64
2022-05-05 10:13:00.648065: W tensorflow/stream_executor/platform/default/

In [12]:
bert.

BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

In [13]:
from keras.datasets import imdb

In [66]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [67]:
X_train[:5]

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369,

In [74]:
X = []
y = []
for line in open('correct_dataset.txt','r').readlines()[1:]:
    line = line.strip().split(",")
    filename = line[0]
    java_file = open(filename, 'r').read().strip()
    X.append(tokenizer.encode(java_file))
    y.append(float(line[1]))

In [85]:
np.random.seed(795)
seq = np.arange(len(X))
np.random.shuffle(sequence)

In [86]:
X = np.array(X)
y = np.array(y)

In [89]:
X = X[seq]
y = y[seq]

no_training_set = int(len(X)*.80)
X_train = X[:no_training_set]
y_train = y[:no_training_set]

X_test = X[no_training_set:]
y_test = y[no_training_set:]

min_y = min(y_train)
max_y = max(y_train)

y_train_normalized = (y_train-min_y) / (max_y-min_y)
y_test_normalized = (y_test-min_y) / (max_y-min_y)

X_train = sequence.pad_sequences(X_train, maxlen=2000)
X_test = sequence.pad_sequences(X_test, maxlen=2000)

In [83]:
len(tokenizer.encode(java_file))

542

In [151]:
embedding_vecor_length = 256
model = Sequential()
model.add(Embedding(10000, embedding_vecor_length, input_length=2000))
model.add(GRU(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])
print(model.summary())
model.fit(X_train, y_train_normalized, epochs=1, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test_normalized, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 2000, 256)         2560000   
                                                                 
 gru_1 (GRU)                 (None, 100)               107400    
                                                                 
 dense_18 (Dense)            (None, 1)                 101       
                                                                 
Total params: 2,667,501
Trainable params: 2,667,501
Non-trainable params: 0
_________________________________________________________________
None
33/33 [==============================] - 76s 2s/step - loss: 0.0409 - root_mean_squared_error: 0.2021
Accuracy: 2.79%


In [109]:
print("MSE", mean_squared_error(y_train, model.predict(X_train)))

MSE 16.0749894614877


In [112]:
print("MSE", mean_squared_error(y_test,model.predict(X_test)))

MSE 7.789438045830727


In [152]:
print("r2_score", r2_score(y_test_normalized, model.predict(X_test), multioutput='variance_weighted'))

r2_score -0.026249084032138992


In [117]:
rmse = RMSE(dtype='float64')

In [153]:
rmse(y_test_normalized, model.predict(X_test))

<tf.Tensor: shape=(), dtype=float64, numpy=0.19683269006064183>

In [121]:
np.std(y_test_normalized)

0.027491602047015982